### 1. Попробуйте изменить параметры нейронной сети работающей с датасетом imdb так, чтобы улучшить ее точность. Приложите анализ.

Я сильно изменял каждый параметр и смотрел, что изменилось при обучении, затем возвращал в изначальное положение и пробовал снова, вот результат этого небольшого анализа? <br>
1) Проверим гипотезу о том, что по первым словам можно понять положительный это отзыв или отрицательный. Я изменил максимальную длину, уменьшил её до 50, обучаться стало в полтора раза быстрее, но качество упало на 1.5%.<br>
2) Батч сайз влияет всё так же - чем он меньше, тем дольше обучается модель, но тем лучше результат. Уменьшение до 25 дало прирост точности до 0.8276.<br>
3) Cлои Embedding имеет 2 параметра - размер словаря (max_features) и размер векторного представления. Я предполагаю, что увеличение размерности векторного представления увеличит смысловую нагрузку, которые эти векторы смогут нести, т.е. слова станут более осмысленные для модели. Однако, хоть модель и обучалась гораздо дольше, качество у меня упало. Так же проверим, что размер словаря может быть небольшим, т.к. нам не надо большинство слов-связок. Я понизил max_features в 1- раз и качество снизилось всего на 1%.<br>
4) Увеличение размера LSTM ячейки где-то в 2 раза увеличило время обучения где-то в 3 раза, при этом качество увеличилось на тестовом прогоне до 0.8275.<br>
5) Дропаут в LSTM помогает избежать переобучения, но высокое значение может помешать в принципе обучить модель. При дропауте в 0.5 ничего особо не изменилось, ни точность, ни скорость. А вот такое же качество recurrent_dropout увеличило точность на 1%.<br>
6) В последнем слое я поменял функцию активации с sigmoid на relu. В результате точность упала на 4%. Полагаю, что вид sigmoid (плавный от 0 до 1) больше подходит для использовании в выходном слое.<br>
7) Оптимайзер SGD быстрее сработал, чем adam, но выдал результат около 0.5. Полагаю, что он не успел даже начать двигаться в нужном направлении, возможно, надо увеличить число эпох.

### 2. Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.

In [31]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 12, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 150, 256
NUM_ITERATIONS = 10 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 5
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/5
158771/158771 [==============================] - 64s 405us/step - loss: 2.3746
Epoch 2/5
158771/158771 [==============================] - 64s 404us/step - loss: 1.9050
Epoch 3/5
158771/158771 [==============================] - 62s 392us/step - loss: 1.7025
Epoch 4/5
158771/158771 [==============================] - 63s 398us/step - loss: 1.5612
Epoch 5/5
158771/158771 [==============================] - 66s 413us/step - loss: 1.45270s - loss: 1.45 - ETA: 0s - l
Генерация из посева: bbing of the
bbing of the court, and she was so they was a little she was a great did all the thing was the door of the court==================================================
Итерация #: 1
Epoch 1/5
158771/158771 [==============================] - 66s 418us/step - loss: 1.3676
Epoch 2/5
158771/158771 [==============================] - 66s 413us/step - loss: 1.2941
Epoch 3/5
158771/158771 [==============================] - 65s 412us/step - loss: 1.2305
Epoch 4/5
158771/158771 [=========

### Результат:
are put out here? she asked. yes, thats it, said the dodo solemnly presented the three gardeners, who was a very

### 3. Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM

Где-то полгода назад я вдоль и поперёк прочёл одну из книг, которые Вы советовали, а именно "Грокаем Глубокое Обучение", и последние главы книги посвещены созданию собственного фреймворка глубокого обучения, похожего на pytorch. Вот часть этого фремворка, большую часть я вырезал, оставил то, что нужно для реализации ячейки LSTM.

In [1]:
import numpy as np

In [2]:
class Tensor(object):
    def __init__(self, data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        if (id is None):
            id = np.random.randint(0, 100000)
        self.id = id
        
        if creators is not None:
            for c in creators:
                if self.id not in c.children:
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
    
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if (cnt != 0):
                return False
        return True
    
    def backward(self, grad=None, grad_origin=None):
        if self.autograd:
            if grad is None:
                grad = Tensor(np.ones_like(self.data))
            if grad_origin is not None:
                if self.children[grad_origin.id] == 0:
                    return
                    raise Exception("Cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
            if self.grad is None:
                self.grad = grad
            else:
                self.grad += grad
            
            if self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None):        
                if self.creation_op == "add":
                    self.creators[0].backward(grad, self)
                    self.creators[1].backward(grad, self)
                if self.creation_op == "neg":
                    self.creators[0].backward(self.grad.__neg__())
                if self.creation_op == "sub":
                    new = Tensor(self.grad.data)
                    self.creators[0].backward(new, self)
                    new = Tensor(self.grad.__neg__().data)
                    self.creators[1].backward(new, self)
                if self.creation_op == "mul":
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new, self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)
                if self.creation_op == "mm":
                    act = self.creators[0]
                    weights = self.creators[1]
                    new = self.grad.mm(weights.transpose())
                    act.backward(new)
                    new = self.grad.transpose().mm(act).transpose()
                    weights.backward(new)
                if self.creation_op == "transpose":
                    self.creators[0].backward(self.grad.transpose())
                if "sum" in self.creation_op:
                    dim = int(self.creation_op.split("_")[1])
                    ds = self.creators[0].data.shape[dim]
                    self.creators[0].backward(self.grad.expand(dim, ds))
                if "expand" in self.creation_op:
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                if self.creation_op == "sigmoid":
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones-self)))
                if self.creation_op == "tanh":
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
                if self.creation_op == "index_select":
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))
                if self.creation_op == "cross_entropy":
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
        
    def __add__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if self.autograd:
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
        
    def __sub__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
        
    def __mul__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)
        
    def sum(self, dim):
        if self.autograd:
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
        
    def expand(self, dim, copies):
        trans_cmd = list(range(0, len(self.data.shape)))
        trans_cmd.insert(dim, len(self.data.shape))
        new_shape = list(self.data.shape) + [copies]
        new_data = self.data.repeat(copies).reshape(new_shape)
        new_data = new_data.transpose(trans_cmd)
        if self.autograd:
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if self.autograd:
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if self.autograd:
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self, x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def sigmoid(self):
        if self.autograd:
            return Tensor(1 / (1 + np.exp(-self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op="sigmoid"
                         )
        return Tensor(1 / (1 + np.exp(-self.data)))
    
    def tanh(self):
        if self.autograd:
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="tanh"
                         )
        return Tensor(np.tanh(self.data))
    
    def index_select(self, indices):
        if self.autograd:
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op="index_select"
                        )
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])
    
    def cross_entropy(self, target_indices):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp, axis=len(self.data.shape)-1, keepdims=True)
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
        
        if self.autograd:
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op="cross_entropy"
                        )
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out
        return Tensor(loss)
    
    def softmax(self):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        return softmax_output
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [3]:
class Layer(object):
    def __init__(self):
        self.parameters = list()
    def get_parameters(self):
        return self.parameters

In [4]:
class Linear(Layer):
    def __init__(self, n_inputs, n_outputs, bias=True):
        super().__init__()
        self.use_bias = bias
        W = np.random.randn(n_inputs, n_outputs)*np.sqrt(2.0/(n_inputs))
        self.weight = Tensor(W, autograd=True)
        if (self.use_bias):
            self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        
        if (self.use_bias):
            self.parameters.append(self.bias)
    
    def forward(self, input):
        if (self.use_bias):
            return input.mm(self.weight)+self.bias.expand(0, len(input.data))
        return input.mm(self.weight)

In [5]:
class SGD(object):
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha
    
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
            
    def step(self, zero=True):
        for p in self.parameters:
            p.data -= self.alpha * p.grad.data
            if zero:
                p.grad.data *= 0        

In [6]:
class Embedding(Layer):
    def __init__(self, vocab_size, dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.dim = dim
        weight = (np.random.rand(vocab_size, dim) - 0.5) / dim
        self.weight = Tensor(weight, autograd=True)
        self.parameters.append(self.weight)
    
    def forward(self, input):
        return self.weight.index_select(input)

In [7]:
class CrossEntropyLoss(Layer):
    def __init__(self):
        super().__init__()
        
    def forward(self, input, target):
        return input.cross_entropy(target)

In [8]:
class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.sigmoid()

In [9]:
class Tanh(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.tanh()

In [10]:
class LSTMCell(Layer):
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        self.xf = Linear(n_inputs, n_hidden)
        self.xi = Linear(n_inputs, n_hidden)
        self.xo = Linear(n_inputs, n_hidden)
        self.xc = Linear(n_inputs, n_hidden)
        self.hf = Linear(n_hidden, n_hidden, bias=False)
        self.hi = Linear(n_hidden, n_hidden, bias=False)
        self.ho = Linear(n_hidden, n_hidden, bias=False)
        self.hc = Linear(n_hidden, n_hidden, bias=False)
        
        self.w_ho = Linear(n_hidden, n_output, bias=False)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xc.get_parameters()
        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()
        self.parameters += self.ho.get_parameters()
        self.parameters += self.hc.get_parameters()
        self.parameters += self.w_ho.get_parameters()
        
    def forward(self, input, hidden):
        prev_hidden = hidden[0]
        prev_cell = hidden[1]
        
        f=(self.xf.forward(input)+self.hf.forward(prev_hidden)).sigmoid()
        i=(self.xi.forward(input)+self.hi.forward(prev_hidden)).sigmoid()
        o=(self.xo.forward(input)+self.ho.forward(prev_hidden)).sigmoid()
        g = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()
        c = (f*prev_cell) + (i*g)
        h = o*c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h, c)
    
    def init_hidden(self, batch_size=1):
        h = Tensor(np.zeros((batch_size, self.n_hidden)), autograd=True)
        c = Tensor(np.zeros((batch_size, self.n_hidden)), autograd=True)
        h.data[:, 0] += 1
        c.data[:, 0] += 1
        return (h, c)

In [18]:
import sys

In [19]:
f = open("alice_in_wonderland.txt", 'r', encoding='utf-8')
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i
indices = np.array(list(map(lambda x:word2index[x], raw)))

embed = Embedding(vocab_size=len(vocab), dim=512)
model = LSTMCell(n_inputs=512, n_hidden=512, n_output=len(vocab))
model.w_ho.weight.data *= 0
criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters()+embed.get_parameters(), alpha=0.05)

batch_size = 16
bptt = 25
n_batches = int(indices.shape[0] / batch_size)

trimmed_indices = indices[:n_batches*batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches)
batched_indices = batched_indices.transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int((n_batches-1) / bptt)
input_batches = input_batched_indices[:n_bptt*bptt]
input_batches = input_batches.reshape(n_bptt, bptt, batch_size)
target_batches = target_batched_indices[:n_bptt*bptt]
target_batches = target_batches.reshape(n_bptt, bptt, batch_size)

def train(iterations=10):
    min_loss = 1000
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0
        
        hidden = model.init_hidden(batch_size=batch_size)
        batches_to_train = len(input_batches)
        
        for batch_i in range(batches_to_train):
            hidden = (Tensor(hidden[0].data, autograd=True),
                      Tensor(hidden[1].data, autograd=True))

            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)
                target = Tensor(target_batches[batch_i][t], autograd=True)
                batch_loss = criterion.forward(output, target)
                if t == 0:
                    losses.append(batch_loss)
                else:
                    losses.append(batch_loss + losses[-1])
            loss = losses[-1]
            
            loss.backward()
            optim.step()
            
            total_loss += loss.data / bptt
            epoch_loss = np.exp(total_loss / (batch_i+1))
            print(min_loss)
            if epoch_loss < min_loss:
                min_loss = epoch_loss
                print()
            log = f"\rIter: {iter}"
            log += f" - Alpha {str(optim.alpha)[0:5]}"
            log += f" - Batch {batch_i+1}/{len(input_batches)}"
            log += f" - Min Loss: {str(min_loss)[0:5]}"
            log += f" - Loss: {epoch_loss}"
            if batch_i == 0:
                log += " - " + generate_sample(70, 'T').replace('\n', ' ')
            sys.stdout.write(log)
        optim.alpha *= 0.99
        print()

def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor([word2index[init_char]])
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()
        
        m = output.data.argmax()
        c = vocab[m]
        input = Tensor([m])
        s += c
    return s

In [20]:
train(1) # для скорости, обучать надо гораздо больше, итераций 100

1000

Iter: 0 - Alpha 0.05 - Batch 1/409 - Min Loss: 85.99 - Loss: 85.99999999999982 - oo                        hfffDBjoooooooo                             85.99999999999982
Iter: 0 - Alpha 0.05 - Batch 2/409 - Min Loss: 85.99 - Loss: 86.0001919677186685.99999999999982

Iter: 0 - Alpha 0.05 - Batch 3/409 - Min Loss: 85.98 - Loss: 85.9822391118870185.98223911188701

Iter: 0 - Alpha 0.05 - Batch 4/409 - Min Loss: 85.95 - Loss: 85.9548203156825185.95482031568251

Iter: 0 - Alpha 0.05 - Batch 5/409 - Min Loss: 85.90 - Loss: 85.902958807145685.9029588071456

Iter: 0 - Alpha 0.05 - Batch 6/409 - Min Loss: 85.78 - Loss: 85.7856351769521785.78563517695217

Iter: 0 - Alpha 0.05 - Batch 7/409 - Min Loss: 85.51 - Loss: 85.5140117644676785.51401176446767

Iter: 0 - Alpha 0.05 - Batch 8/409 - Min Loss: 84.96 - Loss: 84.9655543900428784.96555439004287

Iter: 0 - Alpha 0.05 - Batch 9/409 - Min Loss: 83.57 - Loss: 83.5791042178419683.57910421784196

Iter: 0 - Alpha 0.05 - Batch 10/409 - Min Loss: 80.

Iter: 0 - Alpha 0.05 - Batch 83/409 - Min Loss: 30.28 - Loss: 30.2825990812049730.28259908120497

Iter: 0 - Alpha 0.05 - Batch 84/409 - Min Loss: 30.08 - Loss: 30.08294720407613530.082947204076135

Iter: 0 - Alpha 0.05 - Batch 85/409 - Min Loss: 29.88 - Loss: 29.88100790213148629.881007902131486

Iter: 0 - Alpha 0.05 - Batch 86/409 - Min Loss: 29.71 - Loss: 29.711191557570729.7111915575707

Iter: 0 - Alpha 0.05 - Batch 87/409 - Min Loss: 29.56 - Loss: 29.56141420563840729.561414205638407

Iter: 0 - Alpha 0.05 - Batch 88/409 - Min Loss: 29.43 - Loss: 29.4343184628692129.43431846286921

Iter: 0 - Alpha 0.05 - Batch 89/409 - Min Loss: 29.20 - Loss: 29.20866480526646629.208664805266466

Iter: 0 - Alpha 0.05 - Batch 90/409 - Min Loss: 29.01 - Loss: 29.01765813570825329.017658135708253

Iter: 0 - Alpha 0.05 - Batch 91/409 - Min Loss: 28.91 - Loss: 28.91257799842601628.912577998426016

Iter: 0 - Alpha 0.05 - Batch 92/409 - Min Loss: 28.77 - Loss: 28.77782025269426628.777820252694266

Iter: 0 

Iter: 0 - Alpha 0.05 - Batch 165/409 - Min Loss: 23.14 - Loss: 23.14827937952868523.148279379528685

Iter: 0 - Alpha 0.05 - Batch 166/409 - Min Loss: 23.09 - Loss: 23.09287822788470723.092878227884707

Iter: 0 - Alpha 0.05 - Batch 167/409 - Min Loss: 23.06 - Loss: 23.06549520187888523.065495201878885

Iter: 0 - Alpha 0.05 - Batch 168/409 - Min Loss: 23.02 - Loss: 23.0251248584070623.02512485840706

Iter: 0 - Alpha 0.05 - Batch 169/409 - Min Loss: 22.97 - Loss: 22.97837436460518422.978374364605184

Iter: 0 - Alpha 0.05 - Batch 170/409 - Min Loss: 22.92 - Loss: 22.92648297423329522.926482974233295

Iter: 0 - Alpha 0.05 - Batch 171/409 - Min Loss: 22.86 - Loss: 22.86520718438011822.865207184380118

Iter: 0 - Alpha 0.05 - Batch 172/409 - Min Loss: 22.81 - Loss: 22.81389646607698722.813896466076987

Iter: 0 - Alpha 0.05 - Batch 173/409 - Min Loss: 22.76 - Loss: 22.76271739784147722.762717397841477

Iter: 0 - Alpha 0.05 - Batch 174/409 - Min Loss: 22.69 - Loss: 22.6959328443422122.6959328443

Iter: 0 - Alpha 0.05 - Batch 246/409 - Min Loss: 20.09 - Loss: 20.09991814927855520.099918149278555

Iter: 0 - Alpha 0.05 - Batch 247/409 - Min Loss: 20.07 - Loss: 20.07078886250372620.070788862503726

Iter: 0 - Alpha 0.05 - Batch 248/409 - Min Loss: 20.06 - Loss: 20.069167826707820.0691678267078
Iter: 0 - Alpha 0.05 - Batch 249/409 - Min Loss: 20.06 - Loss: 20.0890462876021320.0691678267078
Iter: 0 - Alpha 0.05 - Batch 250/409 - Min Loss: 20.06 - Loss: 20.08113309803974620.0691678267078

Iter: 0 - Alpha 0.05 - Batch 251/409 - Min Loss: 20.04 - Loss: 20.04949275438167820.049492754381678

Iter: 0 - Alpha 0.05 - Batch 252/409 - Min Loss: 20.01 - Loss: 20.01829572603667620.018295726036676

Iter: 0 - Alpha 0.05 - Batch 253/409 - Min Loss: 19.98 - Loss: 19.98665670769971619.986656707699716

Iter: 0 - Alpha 0.05 - Batch 254/409 - Min Loss: 19.95 - Loss: 19.95438804658179419.954388046581794

Iter: 0 - Alpha 0.05 - Batch 255/409 - Min Loss: 19.92 - Loss: 19.92822169945914719.928221699459147

I

Iter: 0 - Alpha 0.05 - Batch 327/409 - Min Loss: 18.51 - Loss: 18.51745991541423618.517459915414236

Iter: 0 - Alpha 0.05 - Batch 328/409 - Min Loss: 18.51 - Loss: 18.51090785437696318.510907854376963

Iter: 0 - Alpha 0.05 - Batch 329/409 - Min Loss: 18.50 - Loss: 18.50126867856504718.501268678565047

Iter: 0 - Alpha 0.05 - Batch 330/409 - Min Loss: 18.47 - Loss: 18.4747336182115618.47473361821156

Iter: 0 - Alpha 0.05 - Batch 331/409 - Min Loss: 18.45 - Loss: 18.45142861635042718.451428616350427

Iter: 0 - Alpha 0.05 - Batch 332/409 - Min Loss: 18.44 - Loss: 18.4447675386504818.44476753865048

Iter: 0 - Alpha 0.05 - Batch 333/409 - Min Loss: 18.42 - Loss: 18.4255973981064418.42559739810644

Iter: 0 - Alpha 0.05 - Batch 334/409 - Min Loss: 18.40 - Loss: 18.4060525007668118.40605250076681

Iter: 0 - Alpha 0.05 - Batch 335/409 - Min Loss: 18.38 - Loss: 18.389140791295518.3891407912955

Iter: 0 - Alpha 0.05 - Batch 336/409 - Min Loss: 18.36 - Loss: 18.3647658740023618.36476587400236

Iter

In [29]:
print(generate_sample(n=500, init_char='\n'))






‘I wher wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout therer wherer fout 


### 4. Предложите свои варианты решения проблемы исчезающего градиента в RNN

Опять же основываясь на книге - затухающий градиент возникает из-за особенностей производной функции активации. К примеру функция сигмоиды имеет плоские хвосты с маленькой производной, и при обратном распространении ошибки, если возникает значение, близкое к 0 или 1, мы получим маленькое значение градиента, и с каждым слоем, в каждым умножением на производную, градиент будет только уменьшаться. Решение сразу видится такое - использовать другую функцию активации, без таких хвостов с малой производной (к примеру ReLu), но тогда может возникнуть другая проблема - взрывной рост градиента (умножение по нескольким слоям производных, каждая из которых >1, будет с каждым разом увеличивать градиент). Взрывной рост тогда можно в ручную ограничить и просто дольше обучать. Можно, наверное, ещё попробовать чередовать по слоям функции активации.
Другое решение - поменять архитектуру чтобы было меньше умножений производных. К примеру, уменьшить число слоёв, либо сделать так, чтобы не пришлось умножать на производную функции активации. Как я онимаю, подход LSTM как раз в этом и заключается - мы добавляем гейты, которые комбинированно влияют на входной сигнал (добавляют новые пути распространения сигнала по разным правилам), и обратное распространение изменяется от простого вычисления производных и умножения матриц на более сложную, зависящую и от сигнала, и от предыдущего какого-то внутреннего состояния ячейки. Комбинация нескольких гейтов приводит к тому, что в обратном распространении сигнал не будет затухать хотя бы в одном гейте, а значит и градиент, пройдя через LSTM ячейку, не будет затухать.